# 🤖 Semantic Kernel Personal Assistant Agent - Code Flow Explanation

This notebook demonstrates how to build an AI agent using Microsoft's Semantic Kernel framework. Here's a step-by-step breakdown of how the entire system works:

## 🏗️ **Architecture Overview**
```
User Input → PersonalAssistantAgent → Semantic Kernel → Azure OpenAI → Response
                    ↓
              Plugin Functions (Weather, Tasks, Utilities)
```

## 📋 **Step-by-Step Code Flow**

### **Step 1: Initial Setup & Imports** 🔧
- Import necessary libraries: `semantic_kernel`, `asyncio`, Azure connectors
- Set up environment variables for Azure OpenAI credentials
- Define configuration constants

### **Step 2: PersonalAssistantAgent Class Initialization** 🎯
When `PersonalAssistantAgent()` is created:
1. **Creates Semantic Kernel instance** - The brain of our agent
2. **Initializes chat history** - Stores conversation context
3. **Calls `setup_kernel()`** - Connects to Azure OpenAI
4. **Calls `setup_plugins()`** - Loads available functions

### **Step 3: Kernel Setup** ⚙️
`setup_kernel()` method:
1. Creates `AzureChatCompletion` service connection
2. Configures endpoint, API key, and deployment name
3. Adds the service to the kernel
4. Enables the agent to communicate with Azure OpenAI

### **Step 4: Plugin Registration** 🔌
`setup_plugins()` method adds three plugin classes:
- **UtilityPlugin**: Math calculations, date/time
- **WeatherPlugin**: Weather information (mock)
- **TaskManagerPlugin**: Task management system

### **Step 5: Plugin Function Definitions** 🛠️
Each plugin contains functions decorated with `@kernel_function`:
- **Function metadata**: Description, name, parameter types
- **Automatic discovery**: Kernel can call these functions when needed
- **Type annotations**: Help the AI understand what each function does

### **Step 6: Chat Processing Flow** 💬
When `agent.chat(user_input)` is called:

1. **Add user message** to chat history
2. **Create system prompt** with agent instructions
3. **Configure execution settings**:
   - Max tokens, temperature
   - **Function calling enabled** (`FunctionChoiceBehavior.Auto()`)
4. **Prepare message array**:
   - System prompt
   - Full chat history
5. **Send to Azure OpenAI** with function calling capability
6. **AI decides** whether to:
   - Respond directly, OR
   - Call one or more plugin functions first
7. **Extract response** and add to chat history
8. **Return final response** to user

## 🧠 **How Function Calling Works**

When the AI receives a user query:

1. **Analyzes the request**: "Calculate 15 * 24 + 100"
2. **Identifies needed function**: `UtilityPlugin.calculate`
3. **Calls the function** with extracted parameters
4. **Gets function result**: "Result: 460"
5. **Incorporates result** into natural language response
6. **Returns to user**: "The calculation gives us 460"

## 🔄 **Example Execution Flow**

**User**: "Add a task: Buy groceries"
1. Input goes to `chat()` method
2. AI recognizes this needs task management
3. Calls `TaskManagerPlugin.add_task("Buy groceries")`
4. Function returns: "✅ Task added: Buy groceries (ID: 1)"
5. AI responds: "I've added 'Buy groceries' to your task list!"

## 🎯 **Key Features Demonstrated**

- **Multi-turn conversations** with memory
- **Function calling** - AI automatically uses tools
- **Plugin architecture** - Modular, extensible design
- **Error handling** - Graceful failure management
- **Type safety** - Annotated parameters for reliability

## 🚀 **Main Demo Function**

The `main()` function runs a simulation:
1. Creates agent instance
2. Runs through predefined test queries
3. Demonstrates various capabilities
4. Shows conversation flow with delays

This creates a complete AI assistant that can handle general chat, perform calculations, manage tasks, and provide weather information - all through natural language interaction! 🌟

In [ ]:
# import azure.ai.projects.models as M

# # show all classes in the module in vertical format
# for name in dir(M):   
#     if isinstance(getattr(M, name), type):
#         print(name)

# #help(M)
# #print(M.ChatCompletionsClient.__doc__)
# #print(M.ExternalTermination.__module__)

In [ ]:
# Single Agent End-to-End Solution using Semantic Kernel
import asyncio
import os
from typing import Annotated
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.functions import kernel_function
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
import json
from datetime import datetime

# Configuration - Set your Azure OpenAI credentials
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT", "your-endpoint-here")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY", "your-api-key-here")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4")

class PersonalAssistantAgent:
    def __init__(self):
        self.kernel = sk.Kernel()
        self.chat_history = ChatHistory()
        self.setup_kernel()
        self.setup_plugins()
        
    def setup_kernel(self):
        """Initialize the kernel with Azure OpenAI service"""
        try:
            # Add Azure OpenAI chat completion service
            azure_chat_service = AzureChatCompletion(
                endpoint=AZURE_OPENAI_ENDPOINT,
                api_key=AZURE_OPENAI_API_KEY,
                deployment_name=AZURE_OPENAI_DEPLOYMENT_NAME,
                service_id="azure_openai"
            )
            self.kernel.add_service(azure_chat_service)
            print("✅ Kernel initialized with Azure OpenAI service")
        except Exception as e:
            print(f"❌ Error setting up kernel: {e}")
            
    def setup_plugins(self):
        """Setup plugins/functions for the agent"""
        # Add built-in plugins
        self.kernel.add_plugin(UtilityPlugin(), plugin_name="Utility")
        self.kernel.add_plugin(WeatherPlugin(), plugin_name="Weather")
        self.kernel.add_plugin(TaskManagerPlugin(), plugin_name="TaskManager")
        print("✅ Plugins loaded successfully")
        
    async def chat(self, user_input: str) -> str:
        """Process user input and return response"""
        try:
            # Add user message to history
            self.chat_history.add_user_message(user_input)
            
            # Create system prompt for the agent
            system_prompt = """
            You are a helpful personal assistant agent built with Semantic Kernel.
            You have access to various tools and functions to help users with:
            - General questions and conversations
            - Weather information
            - Task management
            - Utility functions (calculations, etc.)
            
            Use the available functions when appropriate and provide helpful, accurate responses.
            Be conversational and friendly in your interactions.
            """
            
            # Get chat completion with function calling
            chat_completion_service = self.kernel.get_service(type=AzureChatCompletion)
            
            # Prepare execution settings with function calling
            execution_settings = chat_completion_service.get_prompt_execution_settings_class()(
                service_id="azure_openai",
                max_tokens=1000,
                temperature=0.7,
                function_choice_behavior=sk.functions.FunctionChoiceBehavior.Auto()
            )
            
            # Create messages including system prompt
            messages = [
                ChatMessageContent(role=AuthorRole.SYSTEM, content=system_prompt)
            ]
            messages.extend(self.chat_history.messages)
            
            # Get response from the model
            response = await chat_completion_service.get_chat_message_contents(
                chat_history=messages,
                settings=execution_settings,
                kernel=self.kernel
            )
            
            # Extract the response content
            assistant_response = str(response[0].content) if response else "I'm sorry, I couldn't process that request."
            
            # Add assistant response to history
            self.chat_history.add_assistant_message(assistant_response)
            
            return assistant_response
            
        except Exception as e:
            error_msg = f"❌ Error processing request: {e}"
            print(error_msg)
            return error_msg

# Plugin Classes
class UtilityPlugin:
    """Utility functions for basic operations"""
    
    @kernel_function(
        description="Calculate mathematical expressions",
        name="calculate"
    )
    def calculate(
        self, 
        expression: Annotated[str, "Mathematical expression to evaluate"]
    ) -> str:
        """Safely evaluate mathematical expressions"""
        try:
            # Simple safe evaluation for basic math
            allowed_chars = set("0123456789+-*/.() ")
            if all(c in allowed_chars for c in expression):
                result = eval(expression)
                return f"Result: {result}"
            else:
                return "Invalid expression. Only basic math operations allowed."
        except Exception as e:
            return f"Calculation error: {e}"
    
    @kernel_function(
        description="Get current date and time",
        name="get_datetime"
    )
    def get_datetime(self) -> str:
        """Get current date and time"""
        return f"Current date and time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

class WeatherPlugin:
    """Weather-related functions"""
    
    @kernel_function(
        description="Get weather information for a location",
        name="get_weather"
    )
    def get_weather(
        self, 
        location: Annotated[str, "City or location name"]
    ) -> str:
        """Get weather information (mock implementation)"""
        # This is a mock implementation - in real scenario, you'd call a weather API
        return f"🌤️ Weather in {location}: Partly cloudy, 22°C (72°F). Light breeze from the west."

class TaskManagerPlugin:
    """Task management functions"""
    
    def __init__(self):
        self.tasks = []
    
    @kernel_function(
        description="Add a new task to the task list",
        name="add_task"
    )
    def add_task(
        self, 
        task: Annotated[str, "Task description"]
    ) -> str:
        """Add a task to the task list"""
        task_id = len(self.tasks) + 1
        task_item = {
            "id": task_id,
            "description": task,
            "created": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            "completed": False
        }
        self.tasks.append(task_item)
        return f"✅ Task added: {task} (ID: {task_id})"
    
    @kernel_function(
        description="List all tasks",
        name="list_tasks"
    )
    def list_tasks(self) -> str:
        """List all tasks"""
        if not self.tasks:
            return "📝 No tasks found."
        
        task_list = "📝 Current tasks:\n"
        for task in self.tasks:
            status = "✅" if task["completed"] else "⏳"
            task_list += f"{status} {task['id']}: {task['description']} (Created: {task['created']})\n"
        return task_list

# Example usage and testing
async def main():
    """Main function to demonstrate the agent"""
    print("🤖 Initializing Personal Assistant Agent...")
    agent = PersonalAssistantAgent()
    
    # Test conversations
    test_queries = [
        "Hello! What can you help me with?",
        "What's the current date and time?",
        "Calculate 15 * 24 + 100",
        "Add a task: Buy groceries for the weekend",
        "Add another task: Schedule dentist appointment",
        "List all my tasks",
        "What's the weather like in London?",
    ]
    
    print("\n🔄 Starting conversation simulation...\n")
    
    for i, query in enumerate(test_queries, 1):
        print(f"👤 User: {query}")
        response = await agent.chat(query)
        print(f"🤖 Assistant: {response}\n")
        
        # Small delay for readability
        await asyncio.sleep(1)

# Run the example
if __name__ == "__main__":
    # Note: Make sure to set your Azure OpenAI credentials before running
    print("🚀 Starting Semantic Kernel Agent Demo...")
    print("⚠️  Make sure to set your Azure OpenAI credentials in environment variables:")
    print("   - AZURE_OPENAI_ENDPOINT")
    print("   - AZURE_OPENAI_API_KEY") 
    print("   - AZURE_OPENAI_DEPLOYMENT_NAME")
    
    # Uncomment to run the demo
    # asyncio.run(main())
    
    print("\n✨ Agent setup complete! Uncomment the last line to run the demo.")